# Tools using the langchain

In [2]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv

load_dotenv()
"""
Initialize the chatbot model

Here we using the ChatGroq class from langchain_groq module to initialize the chatbot model.
The model is llama3-70b-8192 which is trained on the llama3 dataset with 70 billion parameters.
The temperature is set to 0 which means the model will always return the most probable token.
The max_tokens is set to None which means the model will generate as many tokens as it wants.
The timeout is set to None which means the model will not timeout.
The max_retries is set to 2 which means the model will retry 2 times if the request fails.
"""

llm = ChatGroq(model="llama3-70b-8192",temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,)

In [3]:
from langchain.agents import tool


@tool
def get_word_length(word: str) -> int:
    """
    Returns the length of the word
    
    Args:
        word (str): The word to be processed
        
    Returns:
        int: The length of the word
    """
    return len(word)

@tool
def get_word_count(word: str) -> int:
    """
    Returns the count of the words
    
    Args:
        word (str): The word to be processed
        
    Returns:
        int: The count of the words
    """
    word_len = word.split(" ")
    return len(word_len)

get_word_count.invoke("Shathis Kannan")

2

In [4]:
"""
Make the tools available to the chatbot
"""
tools = [get_word_length,get_word_count]

In [5]:
# Importing the ChatPromptTemplate and MessagesPlaceholder from langchain_core.prompts for creating the chat prompt
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

"""
Create the chat prompt

Here we are creating a chat prompt using the ChatPromptTemplate class from langchain_core.prompts module.
The chat prompt is a list of messages that the chatbot will use to interact with the user.
The ChatPromptTemplate.from_messages method is used to create the chat prompt.
The first message is from the system which says "You are very powerful assistant, but don't know current events".
The second message is from the user which is the input message.
The third message is a placeholder for the agent scratchpad which is used to store the intermediate values of the agent.
"""

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [6]:
# bind the tools to the chatbot model
llm_with_tools = llm.bind_tools(tools)

In [7]:
"""
Langchain agent pipeline for the chatbot

Here we are creating a langchain agent pipeline for the chatbot.
The agent pipeline is created using the langchain agent pipeline operator.
The agent pipeline is a composition of the following functions:
1. The input message is passed to the chatbot model.
2. The intermediate steps are passed to the agent scratchpad.
3. The output of the chatbot model is passed to the output parser.
"""
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [8]:
from langchain.agents import AgentExecutor

# Create the agent executor
"""
Create the agent executor

Here we are creating the agent executor for the chatbot.
The agent executor is created using the AgentExecutor class from langchain agents module.
The agent executor is used to execute the agent pipeline.
The agent executor takes the agent and tools as input.
The verbose parameter is set to True which means the agent executor will print the output.
"""
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [9]:
"""
Execute the agent pipeline

Here we are executing the agent pipeline using the agent executor.
The agent executor takes the input message as input.
The agent executor will return the output of the agent pipeline.
"""
list(agent_executor.stream({"input": """
                            Give the word count for the Below paragraph:
                            In this case we're relying on OpenAI tool calling LLMs, which take tools as a separate argument and have been
                            specifically trained to know when to invoke those tools.
                            To pass in our tools to the agent, we just need to format them to the OpenAI tool format and pass them to our model.
                            (By bind-ing the functions, we're making sure that they're passed in each time the model is invoked.)
                            """}))



> Entering new AgentExecutor chain...

Invoking: `get_word_count` with `{'word': "In this case we're relying on OpenAI tool calling LLMs, which take tools as a separate argument and have been specifically trained to know when to invoke those tools. To pass in our tools to the agent, we just need to format them to the OpenAI tool format and pass them to our model. (By bind-ing the functions, we're making sure that they're passed in each time the model is invoked.)"}`


71The word count for the given paragraph is 71.

> Finished chain.


[{'actions': [ToolAgentAction(tool='get_word_count', tool_input={'word': "In this case we're relying on OpenAI tool calling LLMs, which take tools as a separate argument and have been specifically trained to know when to invoke those tools. To pass in our tools to the agent, we just need to format them to the OpenAI tool format and pass them to our model. (By bind-ing the functions, we're making sure that they're passed in each time the model is invoked.)"}, log='\nInvoking: `get_word_count` with `{\'word\': "In this case we\'re relying on OpenAI tool calling LLMs, which take tools as a separate argument and have been specifically trained to know when to invoke those tools. To pass in our tools to the agent, we just need to format them to the OpenAI tool format and pass them to our model. (By bind-ing the functions, we\'re making sure that they\'re passed in each time the model is invoked.)"}`\n\n\n', message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'id': 'cal

In [10]:
"""
Execute the agent pipeline

Here we are executing the agent pipeline using the agent executor.
The agent executor takes the input message as input.
The agent executor will return the output of the agent pipeline.
"""
result = llm.invoke("How many letters in the Shathis Kannan")

list(agent_executor.stream({"input": "How many letters in the Shathis Kannan"}))



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'Shathis'}`


7
Invoking: `get_word_length` with `{'word': 'Kannan'}`


6The total number of letters in "Shathis Kannan" is 7 + 6 = 13.

> Finished chain.


[{'actions': [ToolAgentAction(tool='get_word_length', tool_input={'word': 'Shathis'}, log="\nInvoking: `get_word_length` with `{'word': 'Shathis'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'id': 'call_5tnn', 'function': {'arguments': '{"word":"Shathis"}', 'name': 'get_word_length'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d2651b39-2291-4dd9-a49e-64373f68734a', tool_calls=[{'name': 'get_word_length', 'args': {'word': 'Shathis'}, 'id': 'call_5tnn', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1077, 'output_tokens': 46, 'total_tokens': 1123}, tool_call_chunks=[{'name': 'get_word_length', 'args': '{"word":"Shathis"}', 'id': 'call_5tnn', 'index': None, 'type': 'tool_call_chunk'}])], tool_call_id='call_5tnn')],
  'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'id': 'call_5tnn', 'function': {'arguments': '{"word":"Shathis"}', 'name': 'get_word_length'}, '

In [11]:
print(result.content)

Let me count the letters for you!

Here are the letters in "Shathis Kannan":

1. S
2. H
3. A
4. T
5. H
6. I
7. S
8. K
9. A
10. N
11. N
12. A
13. N

There are 13 letters in "Shathis Kannan".
